In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as ddf
from pandas import Series, DataFrame
%matplotlib inline

In [2]:
import seaborn as sn
import plotly.express as px

## Upload census geometry data

In [5]:
ur_files = ddf.read_csv('county_shp.csv', dtype = {'STATEFP': str, 'COUNTYFP': str, 'TRACTCE': str, 'NAME': str, 'NAMELSAD': str})
census = ur_files.compute()
census.head()

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,NAMELSADCO,STATE_NAME,LSAD,ALAND,AWATER,geometry
0,13,163,960300,1400000US13163960300,13163960300,9603,Census Tract 9603,GA,Jefferson County,Georgia,CT,564728206,4845400,"POLYGON ((-82.661917 33.126331, -82.651937 33...."
1,37,109,071201,1400000US37109071201,37109071201,712.01,Census Tract 712.01,NC,Lincoln County,North Carolina,CT,5976613,6611763,"POLYGON ((-80.993444 35.478784, -80.9914239999..."
2,25,025,050200,1400000US25025050200,25025050200,502,Census Tract 502,MA,Suffolk County,Massachusetts,CT,243271,0,"POLYGON ((-71.040029 42.37865, -71.03921099999..."
3,17,031,230200,1400000US17031230200,17031230200,2302,Census Tract 2302,IL,Cook County,Illinois,CT,214283,0,"POLYGON ((-87.71341700000001 41.913589, -87.71..."
4,41,047,001607,1400000US41047001607,41047001607,16.07,Census Tract 16.07,OR,Marion County,Oregon,CT,1814790,0,"POLYGON ((-122.990318 44.968214, -122.987853 4..."


In [6]:
census.shape

(85230, 14)

In [7]:
from shapely import wkt

In [8]:
def findCentroid(g):
    p1 = wkt.loads(g)
    return p1.centroid.wkt

In [9]:
census['Centroid'] = census['geometry'].apply(findCentroid)

In [10]:
census.head()

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,NAMELSADCO,STATE_NAME,LSAD,ALAND,AWATER,geometry,Centroid
0,13,163,960300,1400000US13163960300,13163960300,9603,Census Tract 9603,GA,Jefferson County,Georgia,CT,564728206,4845400,"POLYGON ((-82.661917 33.126331, -82.651937 33....",POINT (-82.42777455266631 33.02009497835021)
1,37,109,071201,1400000US37109071201,37109071201,712.01,Census Tract 712.01,NC,Lincoln County,North Carolina,CT,5976613,6611763,"POLYGON ((-80.993444 35.478784, -80.9914239999...",POINT (-80.9680558426884 35.45990205837115)
2,25,025,050200,1400000US25025050200,25025050200,502,Census Tract 502,MA,Suffolk County,Massachusetts,CT,243271,0,"POLYGON ((-71.040029 42.37865, -71.03921099999...",POINT (-71.03573713399868 42.37754679162973)
3,17,031,230200,1400000US17031230200,17031230200,2302,Census Tract 2302,IL,Cook County,Illinois,CT,214283,0,"POLYGON ((-87.71341700000001 41.913589, -87.71...",POINT (-87.71008151357118 41.91187356531011)
4,41,047,001607,1400000US41047001607,41047001607,16.07,Census Tract 16.07,OR,Marion County,Oregon,CT,1814790,0,"POLYGON ((-122.990318 44.968214, -122.987853 4...",POINT (-122.9834298654585 44.96158272247879)


## upload temp count data - 2010

In [13]:
ur_files = ddf.read_csv('MaxTemps/MaxTemp_1961.csv', dtype={'station': 'object'})
temp = ur_files.compute()
temp = temp.loc[:, ~temp.columns.str.contains('^Unnamed')]
temp.head()

,STATION,LATITUDE,LONGITUDE,NAME,YEAR,TEMP,MIN,MAX
0,2807099999,68.607269,27.405328,"IVALO, FI",1961,72.4,63.0,86.0
1,2836099999,67.395033,26.619133,"SODANKYLA, FI",1961,74.3,61.0,86.0
2,2864099999,65.781889,24.599100,"KEMI TORNIO, FI",1961,77.2,66.0,88.0
3,2869099999,65.987575,29.239381,"KUUSAMO, FI",1961,74.1,59.0,82.0
4,2875099999,64.930061,25.354564,"OULU, FI",1961,75.6,66.0,86.0


In [14]:
temp['YEAR'].unique()

array([1961])

In [15]:
temp.shape

(3959, 8)

In [16]:
def findLon(point):
    strList = point.split(' ')
    lon = strList[1][1:]
    lon = float(lon)
    return lon

In [17]:
def findLat(point):
    strList = point.split(' ')
    lat = strList[2][:-1]
    lat = float(lat)
    return lat

In [18]:
census['lat'] = census['Centroid'].apply(findLat)
census['lon'] = census['Centroid'].apply(findLon)

In [19]:
census.head()

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,NAMELSADCO,STATE_NAME,LSAD,ALAND,AWATER,geometry,Centroid,lat,lon
0,13,163,960300,1400000US13163960300,13163960300,9603,Census Tract 9603,GA,Jefferson County,Georgia,CT,564728206,4845400,"POLYGON ((-82.661917 33.126331, -82.651937 33....",POINT (-82.42777455266631 33.02009497835021),33.020095,-82.427775
1,37,109,071201,1400000US37109071201,37109071201,712.01,Census Tract 712.01,NC,Lincoln County,North Carolina,CT,5976613,6611763,"POLYGON ((-80.993444 35.478784, -80.9914239999...",POINT (-80.9680558426884 35.45990205837115),35.459902,-80.968056
2,25,025,050200,1400000US25025050200,25025050200,502,Census Tract 502,MA,Suffolk County,Massachusetts,CT,243271,0,"POLYGON ((-71.040029 42.37865, -71.03921099999...",POINT (-71.03573713399868 42.37754679162973),42.377547,-71.035737
3,17,031,230200,1400000US17031230200,17031230200,2302,Census Tract 2302,IL,Cook County,Illinois,CT,214283,0,"POLYGON ((-87.71341700000001 41.913589, -87.71...",POINT (-87.71008151357118 41.91187356531011),41.911874,-87.710082
4,41,047,001607,1400000US41047001607,41047001607,16.07,Census Tract 16.07,OR,Marion County,Oregon,CT,1814790,0,"POLYGON ((-122.990318 44.968214, -122.987853 4...",POINT (-122.9834298654585 44.96158272247879),44.961583,-122.983430


In [20]:
def ID_Coordinate(ID,lat, lon):
    setList = []
    setList.append(ID)
    setList.append(lat)
    setList.append(lon)
    return setList

In [21]:
temp['ID_Coordinate'] = temp.apply(lambda x: ID_Coordinate(x['STATION'], x['LATITUDE'], x['LONGITUDE']), axis=1)

In [22]:
def dist(p0, p1):
    return (((p0[0] - p1[0])**2) + ((p0[1] - p1[1])**2))**.5

In [23]:
pointList = temp['ID_Coordinate'].tolist()

In [24]:
len(pointList)

3959

In [25]:
pointList[:5]

[[2807099999, 68.607269, 27.405328],
 [2836099999, 67.395033, 26.619133],
 [2864099999, 65.781889, 24.5991],
 [2869099999, 65.987575, 29.239381],
 [2875099999, 64.930061, 25.354564]]

In [26]:
def findSquare(pointList, lat, lon):
    square = []
    for point in pointList:
        point_lat = point[1]
        point_lon = point[2]
        if point_lat < lat + 1 and point_lat > lat - 1 and point_lon < lon + 1 and point_lon > lon - 1:
            square.append(point)
        
    return square

In [27]:
result1 = findSquare(pointList,33.020095, -82.427775)
result1

[[94999900193, 33.65, -81.683],
 [99999903820, 33.3644, -81.9633],
 [99999913873, 33.948, -83.3275]]

In [28]:
def shortestDistance(lat, lon):
    p1 = []
    p1.append(lat)
    p1.append(lon)
    distants = []
    square = findSquare(pointList, lat, lon)
    for point in square:
        p_id = point[0]
        p_lat = point[1]
        p_lon = point[2]
        p2 = []
        p2.append(p_lat)
        p2.append(p_lon)
        distant = dist(p1, p2)
        dist_id = []
        dist_id.append(p_id)
        dist_id.append(distant)
        distants.append(dist_id)
    distantList = []    
    for ele in distants:
        distantList.append(ele[1])
    for item in distants:
        if item[1] == min(distantList):
            return item[0]

In [29]:
census['NearestStation'] = census.apply(lambda x: shortestDistance(x['lat'], x['lon']), axis =1)

In [30]:
census = census.rename(columns = {'NearestStation': 'station'})

In [32]:
temp = temp.rename(columns = {'STATION': 'station'})

In [33]:
census_temp = census.merge(temp, on = 'station', how = 'left')
census_temp.head()

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME_x,NAMELSAD,STUSPS,NAMELSADCO,STATE_NAME,...,lon,station,LATITUDE,LONGITUDE,NAME_y,YEAR,TEMP,MIN,MAX,ID_Coordinate
0,13,163,960300,1400000US13163960300,13163960300,9603,Census Tract 9603,GA,Jefferson County,Georgia,...,-82.427775,9.999990e+10,33.36440,-81.96330,"AUGUSTA BUSH FIELD AIRPORT, GA US",1961.0,85.9,73.9,98.1,"[99999903820, 33.3644, -81.9633]"
1,37,109,071201,1400000US37109071201,37109071201,712.01,Census Tract 712.01,NC,Lincoln County,North Carolina,...,-80.968056,9.999991e+10,35.22360,-80.95520,"CHARLOTTE DOUGLAS AIRPORT, NC US",1961.0,82.3,73.9,93.9,"[99999913881, 35.2236, -80.9552]"
2,25,025,050200,1400000US25025050200,25025050200,502,Census Tract 502,MA,Suffolk County,Massachusetts,...,-71.035737,7.250901e+10,42.36060,-71.00970,"BOSTON, MA US",1961.0,82.8,75.0,95.0,"[72509014739, 42.3606, -71.0097]"
3,17,031,230200,1400000US17031230200,17031230200,2302,Census Tract 2302,IL,Cook County,Illinois,...,-87.710082,9.999991e+10,41.78611,-87.75222,"CHICAGO MIDWAY AIRPORT, IL US",1961.0,83.5,75.0,93.9,"[99999914819, 41.78611, -87.75222]"
4,41,047,001607,1400000US41047001607,41047001607,16.07,Census Tract 16.07,OR,Marion County,Oregon,...,-122.983430,9.999992e+10,44.90500,-123.00110,"SALEM AIRPORT MCNARY FIELD, OR US",1961.0,80.2,66.0,102.9,"[99999924232, 44.905, -123.0011]"


In [34]:
census_temp.columns

Index(['STATEFP', 'COUNTYFP', 'TRACTCE', 'AFFGEOID', 'GEOID', 'NAME_x',
       'NAMELSAD', 'STUSPS', 'NAMELSADCO', 'STATE_NAME', 'LSAD', 'ALAND',
       'AWATER', 'geometry', 'Centroid', 'lat', 'lon', 'station', 'LATITUDE',
       'LONGITUDE', 'NAME_y', 'YEAR', 'TEMP', 'MIN', 'MAX', 'ID_Coordinate'],
      dtype='object')

In [35]:
census_temp['fips'] = census_temp['STATEFP'] + census_temp['COUNTYFP']
temp_max = census_temp[['fips', 'YEAR', 'TEMP', 'MIN', 'MAX']]
temp_max.head()

,fips,YEAR,TEMP,MIN,MAX
0,13163,1961.0,85.9,73.9,98.1
1,37109,1961.0,82.3,73.9,93.9
2,25025,1961.0,82.8,75.0,95.0
3,17031,1961.0,83.5,75.0,93.9
4,41047,1961.0,80.2,66.0,102.9


In [37]:
example = temp_max[temp_max['fips'] == '13163']
example

,fips,YEAR,TEMP,MIN,MAX
0,13163,1961.0,85.9,73.9,98.1
67036,13163,1961.0,85.9,73.9,98.1
77290,13163,1961.0,85.9,73.9,98.1
82423,13163,1961.0,85.9,73.9,98.1


In [38]:
temp_max = temp_max.dropna()
temp_max = temp_max.groupby('fips').mean()
temp_max = temp_max.reset_index()
temp_max.head()

,fips,YEAR,TEMP,MIN,MAX
0,01001,1961.0,85.888235,78.411765,95.064706
1,01003,1961.0,83.797674,79.460465,94.067442
2,01005,1961.0,84.700000,75.544444,95.500000
3,01007,1961.0,87.400000,82.950000,96.100000
4,01009,1961.0,83.518750,75.843750,96.100000


In [39]:
temp_max.to_csv('temp_max_county_level_1961.csv')